In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.metrics import accuracy_score
from scipy.stats import mode
import pandas as pd
import numpy as np
import os
import errno
import tensorflow as tf
import copy
from copy import deepcopy

from IPython.display import Markdown, display

from preprocess import preprocess_data
from adversarial_debiasing import adversarial
from prejudice_remover import prejudice
from neuralnet import nondebiased_classifier

from aif360.datasets import BinaryLabelDataset, StructuredDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

In [44]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult()
# train1, test1 are the original dataset
train1, test1 = dataset_orig.split([0.7], shuffle=True)
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(train1)
# dataset_transf_train, test1 are for the reweighed dataset
dataset_transf_train = RW.transform(train1)

In [59]:
# change weights to whole numbers
for i in range(dataset_transf_train.instance_weights.size):
    dataset_transf_train.instance_weights[i] = (round(dataset_transf_train.instance_weights[i] / 0.1) * 0.1) * 10
weights = copy.deepcopy(dataset_transf_train.instance_weights)

In [47]:
# change dataset_transf_train.features according to the weights of each instance
sum_weights = 0
for i in range(dataset_transf_train.features.shape[0]):
    row = copy.deepcopy(dataset_transf_train.features[i])
    row.resize(1,18)
    weight = int(weights[i])
    sum_weights = sum_weights+weight
    for j in range(weight-1):
        dataset_transf_train.features = np.concatenate((dataset_transf_train.features,row))

In [58]:
# dump dataset_transf_train (numpy.ndarray) to csv
np.savetxt("dataset_transf_train_features.csv", dataset_transf_train.features, delimiter=",")

In [52]:
metric_dataset_debiasing_train = []
metric_dataset_debiasing_test = []
metric_dataset_prejudice_train = []
metric_dataset_prejudice_test = []

################## without reweighing ##########################

# adversarial debiasing
sess = tf.Session()
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)
debiased_model.fit(train1)
dataset_debiasing_train = debiased_model.predict(train1)
dataset_debiasing_test = debiased_model.predict(test1)

# prejudice remover
prejudice_model = PrejudiceRemover(eta=100, sensitive_attr='sex')
prejudice_model.fit(train1)
dataset_prejudice_train = prejudice_model.predict(train1)
dataset_prejudice_test = prejudice_model.predict(test1)

sess.close()
tf.reset_default_graph()

# metrics
metric_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_dataset_debiasing_train.append(metric_debiasing_train.mean_difference())
metric_dataset_debiasing_test.append(metric_debiasing_test.mean_difference())

metric_prejudice_train = BinaryLabelDatasetMetric(dataset_prejudice_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_prejudice_test = BinaryLabelDatasetMetric(dataset_prejudice_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_dataset_prejudice_train.append(metric_prejudice_train.mean_difference())
metric_dataset_prejudice_test.append(metric_prejudice_test.mean_difference())

epoch 0; iter: 0; batch classifier loss: 0.689295; batch adversarial loss: 0.826605
epoch 0; iter: 200; batch classifier loss: 1.091300; batch adversarial loss: 1.006659
epoch 1; iter: 0; batch classifier loss: 1.074867; batch adversarial loss: 0.922128
epoch 1; iter: 200; batch classifier loss: 0.985737; batch adversarial loss: 0.789555
epoch 2; iter: 0; batch classifier loss: 1.074876; batch adversarial loss: 0.708658
epoch 2; iter: 200; batch classifier loss: 0.576550; batch adversarial loss: 0.653842
epoch 3; iter: 0; batch classifier loss: 0.485614; batch adversarial loss: 0.662933
epoch 3; iter: 200; batch classifier loss: 0.366582; batch adversarial loss: 0.631648
epoch 4; iter: 0; batch classifier loss: 0.380329; batch adversarial loss: 0.647698
epoch 4; iter: 200; batch classifier loss: 0.339574; batch adversarial loss: 0.659192
epoch 5; iter: 0; batch classifier loss: 0.436314; batch adversarial loss: 0.623152
epoch 5; iter: 200; batch classifier loss: 0.500673; batch adversa

epoch 48; iter: 0; batch classifier loss: 0.420530; batch adversarial loss: 0.591593
epoch 48; iter: 200; batch classifier loss: 0.517643; batch adversarial loss: 0.601277
epoch 49; iter: 0; batch classifier loss: 0.583936; batch adversarial loss: 0.595547
epoch 49; iter: 200; batch classifier loss: 0.464953; batch adversarial loss: 0.610254


In [53]:
print("Without reweighing: ")
print("With adv debiasing: ")
print(metric_dataset_debiasing_train, '\t\t',metric_dataset_debiasing_test)
print("With Prejudice remover: ")
print(metric_dataset_prejudice_train, '\t\t',metric_dataset_prejudice_test)
print('\n\n')

Without reweighing: 
With adv debiasing: 
[-0.07800066931484553] 		 [-0.07669695900538484]
With Prejudice remover: 
[-0.22904440746255583] 		 [-0.2289119804400978]





In [54]:
################## with reweighing ##########################

# adversarial debiasing
sess = tf.Session()
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)
debiased_model.fit(dataset_transf_train)
dataset_debiasing_train = debiased_model.predict(dataset_transf_train)
dataset_debiasing_test = debiased_model.predict(test1)

# prejudice remover
prejudice_model = PrejudiceRemover(eta=100, sensitive_attr='sex')
prejudice_model.fit(dataset_transf_train)
dataset_prejudice_train = prejudice_model.predict(dataset_transf_train)
dataset_prejudice_test = prejudice_model.predict(test1)

sess.close()
tf.reset_default_graph()

# metrics
metric_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_dataset_debiasing_train.append(metric_debiasing_train.mean_difference())
metric_dataset_debiasing_test.append(metric_debiasing_test.mean_difference())

metric_prejudice_train = BinaryLabelDatasetMetric(dataset_prejudice_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_prejudice_test = BinaryLabelDatasetMetric(dataset_prejudice_test, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
metric_dataset_prejudice_train.append(metric_prejudice_train.mean_difference())
metric_dataset_prejudice_test.append(metric_prejudice_test.mean_difference())

IndexError: index 255668 is out of bounds for axis 0 with size 34189